In [1]:
import ee 
import geemap
ee.Initialize()

In [3]:
values= ee.List.sequence(1, 10)
print(values.getInfo())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [3]:
count= values.reduce(ee.Reducer.count())
print(count.getInfo())

10


In [5]:
values.size().getInfo()

10

In [4]:
minimum= values.reduce(ee.Reducer.min())
print(minimum.getInfo())

1


In [4]:
min_max= values.reduce(ee.Reducer.minMax()) 
min_max.getInfo()

{'max': 10, 'min': 1}

In [5]:
mean= values.reduce(ee.Reducer.mean())
print(mean.getInfo())

5.5


In [6]:
std= values.reduce(ee.Reducer.stdDev())
print(std.getInfo())

2.8722813232690143


# ImageCollection reducer

In [7]:
Map= geemap.Map()

collection=(
     ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
     .filterDate('2019-01-01', '2019-12-31')
     .filter(ee.Filter.eq('WRS_PATH', 44))
     .filter(ee.Filter.eq('WRS_ROW', 34))
)

median= collection.reduce(ee.Reducer.median())

vis_params= {
    'bands': ['B5_median', 'B4_median', 'B3_median'],
    'gamma': 2,
}
Map.setCenter(-122.3578, 37.7726, 12)
Map.addLayer(median, vis_params, 'Median Image')
Map

Map(center=[37.7726, -122.3578], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [8]:
collection.size().getInfo()

21

In [10]:

image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318').select(['B4', 'B3', 'B2'])
maxValue = image.reduce(ee.Reducer.max())
Map.centerObject(image, 8)
Map.addLayer(image, {}, 'Original image')
Map.addLayer(maxValue, {'max': 13000}, 'Maximum value image')
Map

Map(bottom=203129.0, center=[37.471602850590656, -122.14451407746262], controls=(WidgetControl(options=['posit…

# Feature Collection Reducations

In [11]:
Map = geemap.Map()
census = ee.FeatureCollection('TIGER/2010/Blocks')
benton = census.filter(
    ee.Filter.And(ee.Filter.eq('statefp10', '41'), ee.Filter.eq('countyfp10', '003'))
)
Map.setCenter(-123.27, 44.57, 13)
Map.addLayer(benton)
Map

Map(center=[44.57, -123.27], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [12]:
properties = ['pop10', 'housing10']
sums = benton.filter(ee.Filter.notNull(properties)).reduceColumns(
    **{'reducer': ee.Reducer.sum().repeat(2), 'selectors': properties}
)
sums

In [14]:
print(benton.aggregate_sum('pop10').getInfo())  
print(benton.aggregate_sum('housing10').getInfo())

85579
36245


In [15]:
benton.aggregate_stats('pop10').getInfo()

{'max': 909,
 'mean': 28.72742531050688,
 'min': 0,
 'sample_sd': 65.02307972064482,
 'sample_var': 4228.000896357331,
 'sum': 85579,
 'sum_sq': 15049451,
 'total_count': 2979,
 'total_sd': 65.01216522963706,
 'total_var': 4226.58162784563,
 'valid_count': 2979,
 'weight_sum': 2979,
 'weighted_sum': 85579}

# Image discription

In [16]:
Map = geemap.Map()

centroid = ee.Geometry.Point([-122.4439, 37.7538])
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(centroid).first()
vis = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(centroid, 8)
Map.addLayer(image, vis, "Landsat-8")
Map
     

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

In [17]:

image.propertyNames()

In [18]:
image.get('CLOUD_COVER')

In [19]:
prop= image.propertyNames()
prop.getInfo()

['IMAGE_QUALITY_TIRS',
 'CLOUD_COVER',
 'system:id',
 'EARTH_SUN_DISTANCE',
 'LANDSAT_ID',
 'system:footprint',
 'system:version',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL',
 'SR_APP_VERSION',
 'SATELLITE',
 'SOLAR_AZIMUTH_ANGLE',
 'IMAGE_QUALITY_OLI',
 'system:time_end',
 'WRS_PATH',
 'system:time_start',
 'SENSING_TIME',
 'ESPA_VERSION',
 'SOLAR_ZENITH_ANGLE',
 'WRS_ROW',
 'GEOMETRIC_RMSE_MODEL_Y',
 'LEVEL1_PRODUCTION_DATE',
 'GEOMETRIC_RMSE_MODEL_X',
 'system:asset_size',
 'PIXEL_QA_VERSION',
 'system:index',
 'system:bands',
 'system:band_names']

In [20]:
stats= geemap.image_stats(image)
stats.getInfo()

{'max': {'B1': 13619,
  'B10': 3157,
  'B11': 3129,
  'B2': 13927,
  'B3': 15027,
  'B4': 15006,
  'B5': 14734,
  'B6': 15394,
  'B7': 15875,
  'pixel_qa': 834,
  'radsat_qa': 254,
  'sr_aerosol': 228},
 'mean': {'B1': 373.17266448937534,
  'B10': 2933.3468360917373,
  'B11': 2916.1595090391606,
  'B2': 427.3772568549278,
  'B3': 614.8436692570979,
  'B4': 591.2399058309231,
  'B5': 1954.2059754383815,
  'B6': 1472.090320001327,
  'B7': 996.8030059412354,
  'pixel_qa': 322.8439734001687,
  'radsat_qa': 0.00016066332799187966,
  'sr_aerosol': 90.67310882142341},
 'min': {'B1': -2000,
  'B10': 2735,
  'B11': 2661,
  'B2': -2000,
  'B3': -1590,
  'B4': -1671,
  'B5': -222,
  'B6': -108,
  'B7': -54,
  'pixel_qa': 322,
  'radsat_qa': 0,
  'sr_aerosol': 8},
 'std': {'B1': 217.8725775842265,
  'B10': 67.60157379721916,
  'B11': 63.8863306004462,
  'B2': 247.29896236085855,
  'B3': 312.21830071192664,
  'B4': 410.7108722666891,
  'B5': 1219.1599281513597,
  'B6': 941.2496196855449,
  'B7': 74